In [50]:
import pandas as pd
import numpy as np
import math
import itertools
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline

def hartigan_wong(A, k, iteration=20): 

    def NC(l): 
        nc = A[IC1==l].shape[0]
        return nc

    def D(i, l): 
        distance = np.sum((centroids[l] - A[i]) ** 2)
        return distance

    # step4 OPTRA-stage
    def step4(IC1, IC2, live, centroids): 
        tmp_live = []
        # for _ in range(iteration): 
        for i in range(m): 
            L1 = IC1[i]

            R2_list = []
            L2_list = []

            if L1 in live: 
                    
                for l in range(k): 

                    # L1以外のクラスターについて計算するため、L1の場合、for文をスキップする
                    if l == L1: 
                        continue

                    R2_list.append((NC(l) * D(i, l) / (NC(l) + 1)))
                    L2_list.append(l)
            else: 
                for l in range(k): 
                    if l in live: 
                        continue
                    R2_list.append((NC(l) * D(i, l) / (NC(l) + 1)))
                    L2_list.append(l)

            # L2がなかったら
            if len(L2_list) == 0: 
                continue

            
            L2 = L2_list[R2_list.index(min(R2_list))]


            # この式を満たす場合、点Iは移動せず、L2がIC2になるだけ
            if min(R2_list) >= (NC(L1) * D(i, L1) / (NC(L1) - 1)): 
                IC2[i] = L2
                # このときは移動が起きていないため、liveに追加しない

            # このとき、点IはクラスターL2に移動する。そして、L1がIC2になる。
            else: 
                IC1[i] = L2
                IC2[i] = L1

                # 点の移動が起きたため、クラスター中心を再計算。
                for l in range(k): 
                    centroids[l] = A[IC1 == l].mean(axis=0)

                # 移動に関わった2つのクラスターはliveに属する    
                if L1 not in tmp_live: 
                    tmp_live.append(L1)
                if L2 not in tmp_live: 
                    tmp_live.append(L2)

        live = tmp_live   
        tmp_live = []

        if len(live) == 0: 
            print('break! because the live set is empty!')
            return IC1, live, centroids
            # break
        else: 
            step6(IC1, IC2, live, centroids)

        return IC1, centroids, live

    # step6 QTRAN stage
    def step6(IC1, IC2, live, centroids):
        for i in range(m): 
            L1 = IC1[i]
            L2 = IC2[i]

            tmp_IC1 = IC1

            R1 = (NC(L1) * D(i, L1)) / (NC(L1) - 1)
            R2 = (NC(L2) * D(i, L2)) / (NC(L2) + 1)

            if R1 >= R2: 
                tmp_IC1[i] = L2
                IC2[i] = L1

                # クラスター中心を更新する
                for l in range(k): 
                    centroids[l] = A[tmp_IC1 == l].mean(axis=0)

        if (IC1 == tmp_IC1).all: 
            IC1 == tmp_IC1
            step4(IC1, IC2, live, centroids)
        else: 
            IC1 = tmp_IC1
            step6(IC1, IC2, live, centroids)

    # インプットのデータより、M(データ数), N(データの次元)を抽出して変数にする
    m = A.shape[0]
    n = A.shape[1]

    # 初期クラスター中心を求める。平均値で並び替える
    # まず、平均値を求める
    ave = np.mean(A, axis = 0)
    # 平均値からの距離を置いておく配列
    d_from_ave = []
    for i in range(m): 
        d_from_ave.append(np.sum((ave - A[i]) ** 2))
    # 平均値からの距離で並び替えたindexを保存。
    idx = np.argsort(d_from_ave)
    centroids = []
    # 該当するindexのデータを初期クラスター中心とする。ん
    for l in range(k): 
        lmk = l * (m / k)
        c_factor = A[idx==lmk] # クラスター中心
        c_factor = list(itertools.chain.from_iterable(c_factor)) # なぜか、二重リストになるので、1重に変更する
        c_factor = np.array(c_factor) # ndarrayに変更
        centroids.append(c_factor)

    # step1 点iが最も近いクラスター中心をIC1, 次に近いものをIC2とする。そして、点iはクラスターIC1に割り当てられる。        
    IC1 = np.full(m, 1000)
    IC2 = np.full(m, 1000)
    # 変更後のIC1を入れておく配列
    tmp_IC1 = np.full(m, 1000)

    for i in range(m): 
        distances = np.sum((centroids - A[i]) ** 2, axis=1)
        IC1[i] = np.argsort(distances)[0]
        IC2[i] = np.argsort(distances)[1]

    # step2 クラスター中心を更新する。
    for l in range(k): 
        centroids[l] = A[IC1 == l].mean(axis=0)

    # step3 最初は、全てのクラスターがthe live setに属する
    live = []
    for l in range(k): 
        live.append(k)
    tmp_live = []


    IC1, centroids, live = step4(IC1, IC2, live, centroids)
    return IC1, centroids, live
def hartigan_missing(data, n_clusters, rs_for_initial_values=0):
    np.random.seed(rs_for_initial_values)
    
    #ランダムにクラスタを割り当てる
    clusters = np.random.randint(0, n_clusters, data.shape[0])

    #nc(クラスタcに属する値の数)
    def nc(cluster) : 
        #nc = len(data[clusters==cluster])
        nc = data[clusters==cluster].shape[0]
        return nc

    #ncj(クラスタcに属する, j列の欠損していない値の数)
    def ncj(cluster, j) : 
        data_cj = data[clusters==cluster][:, j]
        complete_data_cj = ~np.isnan(data_cj)
        ncj = np.count_nonzero(complete_data_cj)
        return ncj

    #x_bar cj 
    def xbar_cj(cluster, j) : 
        sum_xij = 0
        for x in data[clusters==cluster][:, j] : 
            if math.isnan(x) : #xが欠損値nanの場合にTrueを返す
                #sum_xij += 0
                pass
            else : 
                sum_xij += x
        return sum_xij / ncj(cluster, j)
    
    #判別式の右辺のシグマ以降を計算する関数
    def right_sigma(q, i, frm = 0, to = data.shape[1]):
        result = 0;
        for j in range(frm, to):
            if data[i, j] == np.nan : 
                #result += 0
                pass
            else : 
                result += (xbar_cj(q, j) - data[i, j]) ** 2
        return result

    #判別式の左辺のシグマ以降を計算する関数
    def left_sigma(p, i, frm = 0, to = data.shape[1]):
        result = 0
        for j in range(frm, to):
            if data[i, j] == np.nan : 
                #result += 0
                pass
            else : 
                result += (ncj(p, j) ** 2 / ((ncj(p, j) - 1) ** 2)) * ((xbar_cj(p, j) - data[i, j]) ** 2)
        return result
    
    #アルゴリズムの実行
    #個体が一巡する間に入れ替えが起こらなければ終了なので、入れ替えが起こらないときに、カウントする
    cnt = 0
    while cnt < data.shape[0] : 
        #判別式の実行
        for i, x in enumerate(data) : 
            #リストを初期化
            right_list = []
            q_list = []
            #pをxの属するクラスタにする
            p = clusters[i]
            for q in range(n_clusters) : 
                #pの属さないクラスタに対し、計算を行う
                if p != q : 
                    left = ((nc(p) - 1) / nc(p)) * left_sigma(p, i)
                    right = (nc(q) / (nc(q) + 1)) * right_sigma(q, i)
                    if left > right : 
                        #判別式を満たすものをリストに追加する
                        right_list.append(right)
                        q_list.append(q)
            if len(right_list) != 0 : 
                #リストが空でなければ、リストの最小値のクラスタを割り当てる
                clusters[i] = q_list[right_list.index(min(right_list))]
                #クラスタが変更されたため、カウントを０にする
                cnt = 0
            else : 
                #リストが空ならば、クラスタは変更されない。したがって、カウントする
                cnt += 1
    centroids = []
    for j in range(n_clusters): 
        centroids.append(data[clusters == j].mean(axis=0))
    return clusters, centroids

In [51]:
df = pd.read_csv("onlinenewspopularity.csv")
# df = data.drop(columns='url')
# 各カラム名の先頭にスペースがあったため、削除
tmp = []
for i in range(len(df.columns)): 
    tmp.append(df.columns[i].replace(" ", ""))
df.columns = tmp

df = df.drop(columns=['url', 'timedelta'])

# 目的変数を対数変換
target = df['shares']
target_log = np.log(target)

# 対数変換したtargetに、列名をつける
target_log = pd.Series(target_log, name='shares_log')
# データフレームに追加
df = pd.concat([df, target_log], axis=1)


In [52]:
# sharesが1400より大きいかどうかで分ける
tmp = []
# sharesが1400よりも大きい場合、1をtmpにappendする
for i in range(len(df)): 
    if df['shares'][i] > 1400: 
        tmp.append(1)
    else: 
        tmp.append(0)
# 0, 1が入ったリストを、列名をつけてpandas.Seriesにする
shares1400 = pd.Series(tmp, name='shares1400')
# concatで、dfに結合
df = pd.concat([df, shares1400], axis=1)

In [53]:
# dfから、shares, shares_logを削除したものをdf2とする
df2 = df.drop(columns=['shares', 'shares_log'])
df2_short = df2[0:1000]

In [54]:
df_short = df[0:1000]

In [47]:
# 行列の標準化
StandardScaler().fit(df_short)
PCA().fit(df_short)
pca_cor = PCA().transform(df_short)

StandardScaler()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: weekday_is_wednesday, dtype: float64

In [32]:
# xとyの散布図の関数
def scatter_plot_df(X, Y, data=df): 
    plt.scatter(x=data[X], y=data[Y])

    plt.title('Scatter Plot of {} vs {}'.format(X, Y))
    plt.xlabel(X)
    plt.ylabel(Y)
    plt.grid(True)
    plt.show()

In [5]:
f = open("/Users/shojiro/desktop/news/OnlineNewsPopularity.names", "rb")
news_names = f.read()
f.close()
print(news_names)

b"1. Title: Online News Popularity\n\n2. Source Information\n    -- Creators: Kelwin Fernandes (kafc \xe2\x80\x98@\xe2\x80\x99 inesctec.pt, kelwinfc \xe2\x80\x99@\xe2\x80\x99 gmail.com),\n                 Pedro Vinagre (pedro.vinagre.sousa \xe2\x80\x99@\xe2\x80\x99 gmail.com) and\n                 Pedro Sernadela\n   -- Donor: Kelwin Fernandes (kafc \xe2\x80\x99@\xe2\x80\x99 inesctec.pt, kelwinfc '@' gmail.com)\n   -- Date: May, 2015\n\n3. Past Usage:\n    1. K. Fernandes, P. Vinagre and P. Cortez. A Proactive Intelligent Decision\n       Support System for Predicting the Popularity of Online News. Proceedings\n       of the 17th EPIA 2015 - Portuguese Conference on Artificial Intelligence,\n       September, Coimbra, Portugal.\n\n       -- Results: \n          -- Binary classification as popular vs unpopular using a decision\n             threshold of 1400 social interactions.\n          -- Experiments with different models: Random Forest (best model),\n             Adaboost, SVM, KNN